In [108]:
from datetime import datetime, time, date
import os
import dotenv
dotenv.load_dotenv()

import pandas as pd
import pytz

from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame

EASTERN_TIME = pytz.timezone("US/Eastern")
UTC = pytz.utc
MARKET_OPEN = time(9, 30, tzinfo=EASTERN_TIME)
MARKET_CLOSE = time(16, 0, tzinfo=EASTERN_TIME)

In [ ]:
def get_minute_historical_data(client, symbol="SPY"):
    request_params = StockBarsRequest(
        symbol_or_symbols=symbol,
        timeframe=TimeFrame.Minute,
        start=datetime(2025, 1, 1),
        end=datetime(2025, 2, 1),
    )
    
    bars = client.get_stock_bars(request_params)

    if type(symbol) == list:
        return {symbol: bars for symbol in symbol}
    else:
        return bars[symbol]

def construct_df_from_bars(bars, filter_trading_hours=True):
    bars_jsonl = []
    for bar in bars:
        if filter_trading_hours:
            if bar.timestamp.astimezone(EASTERN_TIME).time() < MARKET_OPEN or bar.timestamp.astimezone(EASTERN_TIME).time() > MARKET_CLOSE:
                continue
        bars_jsonl.append({
            "caldt": bar.timestamp.astimezone(EASTERN_TIME),
            "day": bar.timestamp.astimezone(EASTERN_TIME).date(),
            "time": bar.timestamp.astimezone(EASTERN_TIME).time(),
            "open": bar.open,
            "high": bar.high,
            "low": bar.low,
            "close": bar.close,
            "volume": bar.volume,
        })
    return pd.DataFrame.from_records(bars_jsonl, index="caldt")

In [ ]:
def abs_move_from_open(df):
    abs_move_from_open = []
    # For each minute in the day
        # abs_move_from_open = abs((df[now].close / df[now.date].open) - 1)
        #abs_move_from_open.append(abs(df.loc[day]. - df.loc[day].iloc[0].open))

In [85]:
client = StockHistoricalDataClient(
    api_key=os.getenv("ALPACA_PAPER_API_KEY"),
    secret_key=os.getenv("ALPACA_PAPER_API_SECRET"),
    sandbox=False,
)

minute_historical = get_minute_historical_data(client)

minute_historical = construct_df_from_bars(minute_historical)